In [18]:
from preprocessing.preprocessing import DataPreprocessing
from mining.routes import RoutesMining
from hmmlearn import hmm
import datetime
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np 
import pandas as pd

In [78]:
preprocessing = DataPreprocessing("data/sbahn_hamburg.csv")
df = preprocessing.create_df()
df = preprocessing.rename_columns(df)
df = preprocessing.change_datatypes(df)

In [79]:
df['ArrivalDate'] = df['Arrival'].dt.date

In [80]:
stations = df.loc[df['sBahnID'] == 'S1']['Station'].unique()

In [81]:
n_stations = len(stations)

In [82]:
s1 =  df.loc[df['sBahnID'] == 'S1']

In [ ]:
emission_probabilities = {
    station: {
        'boardings': np.random.dirichlet(np.ones(3), size=1)[0],
        'alightings': np.random.dirichlet(np.ones(3), size=1)[0]
    }
    for station in range(n_stations)
}

# Convert the emission probabilities to a matrix format expected by hmmlearn
emission_matrix = np.array([np.hstack((emission_probabilities[station]['boardings'], 
                                       emission_probabilities[station]['alightings']))
                            for station in range(n_stations)])


In [48]:
t = df.loc[(df['sBahnID'] == 'S21') & 
       #(df['TrainID'] == '248206') & 
       (df['ArrivalDate'] >= datetime.date(2016,12,10)) & (df['ArrivalDate'] <= datetime.date(2016,12,11))]

In [49]:
t.to_csv('data/exploration.csv', index=True,  sep=';', encoding='utf-8')

In [50]:
preprocessing.hidden_states(df)

                                        StationSequence
0     Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...
1     Elbgaustraße,Eidelstedt,Stellingen,Langenfelde...
2     Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...
3     Elbgaustraße,Eidelstedt,Stellingen,Langenfelde...
4     Elbgaustraße,Eidelstedt,Stellingen,Langenfelde...
...                                                 ...
1025  Pinneberg,Thesdorf,Halstenbek,Krupunder,Elbgau...
1026  Elbgaustraße,Eidelstedt,Stellingen,Langenfelde...
1027  Elbgaustraße,Eidelstedt,Stellingen,Langenfelde...
1028  Othmarschen,Bahrenfeld,Altona,Königstraße,Reep...
1029  Elbgaustraße,Eidelstedt,Stellingen,Langenfelde...

[1030 rows x 1 columns]


1030

In [53]:
sums= df.groupby(['TrainID', 'sBahnID']).agg({'Boardings':'sum','Alightings':'sum'})

In [58]:
df.groupby(['TrainID', 'sBahnID']).agg({'Boardings':'sum','Alightings':'sum'})

,,Boardings,Alightings
TrainID,sBahnID,,
237033n,S1,1420,1549
237043n,S1,955,1074
237053,S1,3669,3905
237056,S1,3435,3335
237063,S1,4669,4839
...,...,...,...
248956AA,S21A,522,556
248973AAAA,S21A,415,405
248979A,S21A,68,77


In [54]:
sums

,Boardings,Alightings
TrainID,,
237033n,1420,1549
237043n,955,1074
237053,3669,3905
237056,3435,3335
237063,4669,4839
...,...,...
248956AA,522,556
248973AAAA,415,405
248979A,68,77


In [176]:
sums.loc[sums['Boardings'] == sums['Alightings']]

,Boardings,Alightings
TrainID,,
243124k,4573,4573
243305A,159,159
243324n,239,239
247293,11403,11403
247526,8039,8039
247945A,137,137
248160,3206,3206
248166,7169,7169


In [180]:
for i in ['243124k', '243305A', '243324n', '247293', '247526', '247945A', '248160', '248166']:
    print(df.loc[df['TrainID'] == i]['ArrivalDate'].min())
    print(df.loc[df['TrainID'] == i]['ArrivalDate'].max())

2017-01-16
2017-03-29
2017-03-01
2017-03-02
2016-12-24
2016-12-24
2016-12-16
2017-03-28
2016-12-10
2017-03-31
2016-12-17
2016-12-18
2016-12-21
2017-03-23
2016-12-12
2017-03-30


In [178]:
df.loc[df['TrainID'] in ['243124k', '243305A', '243324n', '247293', '247526', '247945A', '248160', '248166'] ]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [57]:
df.loc[df['TrainID'] == '247293']

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,StationSequence
47968,247293,Wedel,74,0,2016-12-16 11:39:07,2016-12-16 12:03:24,WL,S1,2016-12-16,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
47969,247293,Rissen,36,1,2016-12-16 12:07:37,2016-12-16 12:08:05,RI,S1,2016-12-16,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
47970,247293,Sülldorf,19,7,2016-12-16 12:11:02,2016-12-16 12:11:55,SDF,S1,2016-12-16,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
47971,247293,Iserbrook,20,7,2016-12-16 12:13:22,2016-12-16 12:14:09,IS,S1,2016-12-16,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
47972,247293,Blankenese,19,14,2016-12-16 12:16:09,2016-12-16 12:17:52,B,S1,2016-12-16,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
...,...,...,...,...,...,...,...,...,...,...
587193,247293,Barmbek,20,29,2017-03-28 12:57:02,2017-03-28 12:57:48,BAP,S1,2017-03-28,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
587194,247293,Alte Wöhr,4,4,2017-03-28 12:59:15,2017-03-28 12:59:50,AW,S1,2017-03-28,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
587195,247293,Rübenkamp,6,31,2017-03-28 13:00:46,2017-03-28 13:01:28,RP,S1,2017-03-28,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."
587196,247293,Ohlsdorf,14,57,2017-03-28 13:03:09,2017-03-28 13:05:26,OPS,S1,2017-03-28,"Wedel,Rissen,Sülldorf,Iserbrook,Blankenese,Hoc..."


In [61]:
label_encoder = LabelEncoder()
label_encoder = LabelEncoder()
df['StationEncoding'] = label_encoder.fit_transform(df['Station']) 

In [62]:
df

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,StationSequence,StationEncoding
0,248206,Aumühle,0,0,2016-12-10 09:27:26,2016-12-10 09:28:49,AH,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",5
1,248206,Wohltorf,5,0,2016-12-10 09:31:36,2016-12-10 09:31:55,WLF,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",68
2,248206,Reinbek,9,0,2016-12-10 09:34:09,2016-12-10 09:34:32,RBK,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",52
3,248206,Bergedorf,43,5,2016-12-10 09:38:01,2016-12-10 09:38:32,BGS,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",8
4,248206,Nettelnburg,10,7,2016-12-10 09:40:32,2016-12-10 09:40:57,NTB,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",42
...,...,...,...,...,...,...,...,...,...,...,...
610665,243064k,Diebsteich,2,8,2017-03-31 08:13:03,2017-03-31 08:13:41,DT,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",15
610666,243064k,Langenfelde,0,0,2017-03-31 08:14:53,2017-03-31 08:15:28,LST,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",40
610667,243064k,Stellingen,0,2,2017-03-31 08:16:49,2017-03-31 08:17:28,STS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",58
610668,243064k,Eidelstedt,0,0,2017-03-31 08:19:00,2017-03-31 08:19:41,EST,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",17


In [67]:
scaler = MinMaxScaler()
df[['BoardingsNormalized', 'AlightingsNormalized']] = scaler.fit_transform(df[['Boardings', 'Alightings']])

In [68]:
df

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,StationSequence,StationEncoding,BoardingsNormalized,AlightingsNormalized
0,248206,Aumühle,0,0,2016-12-10 09:27:26,2016-12-10 09:28:49,AH,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",5,0.000000,0.000000
1,248206,Wohltorf,5,0,2016-12-10 09:31:36,2016-12-10 09:31:55,WLF,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",68,0.005035,0.000000
2,248206,Reinbek,9,0,2016-12-10 09:34:09,2016-12-10 09:34:32,RBK,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",52,0.009063,0.000000
3,248206,Bergedorf,43,5,2016-12-10 09:38:01,2016-12-10 09:38:32,BGS,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",8,0.043303,0.005247
4,248206,Nettelnburg,10,7,2016-12-10 09:40:32,2016-12-10 09:40:57,NTB,S21,2016-12-10,"Aumühle,Wohltorf,Reinbek,Bergedorf,Nettelnburg...",42,0.010070,0.007345
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610665,243064k,Diebsteich,2,8,2017-03-31 08:13:03,2017-03-31 08:13:41,DT,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",15,0.002014,0.008395
610666,243064k,Langenfelde,0,0,2017-03-31 08:14:53,2017-03-31 08:15:28,LST,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",40,0.000000,0.000000
610667,243064k,Stellingen,0,2,2017-03-31 08:16:49,2017-03-31 08:17:28,STS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",58,0.000000,0.002099
610668,243064k,Eidelstedt,0,0,2017-03-31 08:19:00,2017-03-31 08:19:41,EST,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",17,0.000000,0.000000


In [80]:
df['Boardings'].shift(1)

0          NaN
1          0.0
2          5.0
3          9.0
4         43.0
          ... 
610665     6.0
610666     2.0
610667     0.0
610668     0.0
610669     0.0
Name: Boardings, Length: 610670, dtype: float64

In [153]:
df['on_train'] = 0
df['going_next_station'] = 0

In [97]:
for i in df["TrainID"].unique():
    for date in df.loc[(df['TrainID'] == i)]['ArrivalDate'].unique():
        df['on_train'] = df.loc[(df['TrainID'] == i) & (df['ArrivalDate'] == date)]['Boardings'] - df.loc[(df['TrainID'] == i) & (df['ArrivalDate'] == date)]['Alightings']
        df['going_next_station'] = df.loc[(df['TrainID'] == i)  & (df['ArrivalDate'] == date)]['on_train'].cumsum()


In [ ]:
for row in df.iterrows()

In [118]:
df['ArrivalDate'].values

array([datetime.date(2016, 12, 10), datetime.date(2016, 12, 10),
       datetime.date(2016, 12, 10), ..., datetime.date(2017, 3, 31),
       datetime.date(2017, 3, 31), datetime.date(2017, 3, 31)],
      dtype=object)

In [173]:
for date in df.loc[(df['TrainID'] == '243064k')]['ArrivalDate'].unique():
    df['on_train'] = df.loc[(df['TrainID'] == '243064k') & (df['ArrivalDate'] == date)]['Boardings'] - df.loc[(df['TrainID'] ==  '243064k') & (df['ArrivalDate'] == date)]['Alightings']
    df['going_next_station'] = df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == date)]['on_train'].cumsum()
    df['divide_value'] = df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['going_next_station'].shift(fill_value=1)
    df['percentage_leaving'] = df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['Alightings'] / df['divide_value'] 
    df['calc-prob1'] = 1 - df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['percentage_leaving']
    df['percentage_leaving'] = df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['calc-prob1'].shift(fill_value=1) * df['calc-prob1'] 


In [154]:
df['on_train'] = df.loc[(df['TrainID'] == '243064k') & (df['ArrivalDate'] == datetime.date(2017,3,31))]['Boardings'] - df.loc[(df['TrainID'] ==  '243064k') & (df['ArrivalDate'] == datetime.date(2017,3,31))]['Alightings']

In [166]:
df['going_next_station'] = df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['on_train'].cumsum()

In [161]:
df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['going_next_station'].shift(1, fill_value=0)

610650    0
610651    0
610652    0
610653    0
610654    0
610655    0
610656    0
610657    0
610658    0
610659    0
610660    0
610661    0
610662    0
610663    0
610664    0
610665    0
610666    0
610667    0
610668    0
610669    0
Name: going_next_station, dtype: int64

In [160]:
df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['going_next_station'].shift(1, fill_value=0) + df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['on_train']

610650     55.0
610651     51.0
610652      7.0
610653     16.0
610654    101.0
610655     -9.0
610656     15.0
610657    -79.0
610658   -146.0
610659    -42.0
610660    -11.0
610661      0.0
610662     -8.0
610663     -3.0
610664     -3.0
610665     -6.0
610666      0.0
610667     -2.0
610668      0.0
610669     -1.0
dtype: float64

In [135]:
df.loc[(df['TrainID'] == '243064k')]['ArrivalDate'].unique()

array([datetime.date(2016, 12, 15), datetime.date(2016, 12, 30),
       datetime.date(2017, 2, 1), datetime.date(2017, 2, 2),
       datetime.date(2017, 2, 8), datetime.date(2017, 1, 20),
       datetime.date(2017, 2, 15), datetime.date(2017, 2, 16),
       datetime.date(2017, 2, 23), datetime.date(2017, 2, 24),
       datetime.date(2017, 3, 2), datetime.date(2016, 12, 31),
       datetime.date(2017, 3, 22), datetime.date(2017, 3, 25),
       datetime.date(2017, 3, 31)], dtype=object)

In [145]:
dates = df.loc[(df['TrainID'] == '243064k')]['ArrivalDate'].unique()
t  =df.loc[(df['TrainID'] == '243064k')]
for i in dates:
    print(i, t['ArrivalDate'],  t['ArrivalDate'] == i)

2016-12-15 35620     2016-12-15
35621     2016-12-15
35622     2016-12-15
35623     2016-12-15
35624     2016-12-15
             ...    
610665    2017-03-31
610666    2017-03-31
610667    2017-03-31
610668    2017-03-31
610669    2017-03-31
Name: ArrivalDate, Length: 308, dtype: object 35620      True
35621      True
35622      True
35623      True
35624      True
          ...  
610665    False
610666    False
610667    False
610668    False
610669    False
Name: ArrivalDate, Length: 308, dtype: bool
2016-12-30 35620     2016-12-15
35621     2016-12-15
35622     2016-12-15
35623     2016-12-15
35624     2016-12-15
             ...    
610665    2017-03-31
610666    2017-03-31
610667    2017-03-31
610668    2017-03-31
610669    2017-03-31
Name: ArrivalDate, Length: 308, dtype: object 35620     False
35621     False
35622     False
35623     False
35624     False
          ...  
610665    False
610666    False
610667    False
610668    False
610669    False
Name: ArrivalDate, Length: 3

In [126]:
 df.loc[(df['TrainID'] == '243064k') & (df['ArrivalDate'] == date)]['Boardings'] - df.loc[(df['TrainID'] ==  '243064k')]['Alightings']

35620     NaN
35621     NaN
35622     NaN
35623     NaN
35624     NaN
         ... 
610665   -6.0
610666    0.0
610667   -2.0
610668    0.0
610669   -1.0
Length: 308, dtype: float64

In [172]:
df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['going_next_station'].shift(fill_value=1)

610650      1.0
610651     55.0
610652    106.0
610653    113.0
610654    129.0
610655    230.0
610656    221.0
610657    236.0
610658    157.0
610659     11.0
610660    -31.0
610661    -42.0
610662    -42.0
610663    -50.0
610664    -53.0
610665    -56.0
610666    -62.0
610667    -62.0
610668    -64.0
610669    -64.0
Name: going_next_station, dtype: float64

In [174]:
df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]#['going_next_station']

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,StationSequence,StationEncoding,BoardingsNormalized,AlightingsNormalized,on_train,going_next_station,divide_value,percentage_leaving,calc-prob1
610650,243064k,Neugraben,55,0,2017-03-31 07:26:52,2017-03-31 07:29:24,NRS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",44,0.055388,0.000000,55.0,55.0,1.0,1.000000,1.000000
610651,243064k,Neuwiedenthal,51,0,2017-03-31 07:31:00,2017-03-31 07:31:40,NWS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",46,0.051360,0.000000,51.0,106.0,55.0,1.000000,1.000000
610652,243064k,Heimfeld,28,21,2017-03-31 07:35:50,2017-03-31 07:36:33,HFS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",27,0.028197,0.022036,7.0,113.0,106.0,0.801887,0.801887
610653,243064k,Harburg-Rathaus,81,65,2017-03-31 07:38:06,2017-03-31 07:38:55,HRF,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",24,0.081571,0.068206,16.0,129.0,113.0,0.340624,0.424779
610654,243064k,Harburg,114,13,2017-03-31 07:39:59,2017-03-31 07:40:44,HRS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",23,0.114804,0.013641,101.0,230.0,129.0,0.381972,0.899225
610655,243064k,Wilhelmsburg,47,56,2017-03-31 07:44:30,2017-03-31 07:45:21,WFS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",67,0.047331,0.058762,-9.0,221.0,230.0,0.680283,0.756522
610656,243064k,Veddel,34,19,2017-03-31 07:47:33,2017-03-31 07:48:32,VLS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",63,0.034240,0.019937,15.0,236.0,221.0,0.691481,0.914027
610657,243064k,Hammerbrook,0,79,2017-03-31 07:51:05,2017-03-31 07:53:33,HAB,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",22,0.000000,0.082896,-79.0,157.0,236.0,0.608060,0.665254
610658,243064k,Hauptbahnhof,0,146,2017-03-31 07:55:32,2017-03-31 07:56:51,HS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",26,0.000000,0.153200,-146.0,11.0,157.0,0.046610,0.070064
610659,243064k,Jungfernstieg,9,51,2017-03-31 07:58:47,2017-03-31 07:59:38,JUS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",33,0.009063,0.053515,-42.0,-31.0,11.0,-0.254777,-3.636364


In [164]:
df.loc[(df['TrainID'] == '243064k')  & (df['ArrivalDate'] == datetime.date(2017,3,31))]['on_train'].cumsum()

610650     55.0
610651    106.0
610652    113.0
610653    129.0
610654    230.0
610655    221.0
610656    236.0
610657    157.0
610658     11.0
610659    -31.0
610660    -42.0
610661    -42.0
610662    -50.0
610663    -53.0
610664    -56.0
610665    -62.0
610666    -62.0
610667    -64.0
610668    -64.0
610669    -65.0
Name: on_train, dtype: float64

In [102]:
df.loc[(df['TrainID'] == i)]['Alightings']

593166      0
593167      1
593168      3
593169      2
593170      3
593171     30
593172     40
593173     11
593174    127
593175     19
593176     12
593177      1
593178      9
593179      3
593180      7
593181     21
593182     35
Name: Alightings, dtype: int32

In [85]:
df['on_train'] = df['Boardings'] - df['Alightings']

In [90]:
df['going_next_station'] = df['on_train'].shift(1, fill_value=df['on_train'][0]) + df['on_train']

In [158]:
df.loc[(df['TrainID'] == '243064k')]

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,StationSequence,StationEncoding,BoardingsNormalized,AlightingsNormalized,on_train,going_next_station
35620,243064k,Neugraben,98,0,2016-12-15 07:27:55,2016-12-15 07:29:34,NRS,S3,2016-12-15,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",44,0.098691,0.000000,NaN,0
35621,243064k,Neuwiedenthal,65,3,2016-12-15 07:31:27,2016-12-15 07:31:58,NWS,S3,2016-12-15,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",46,0.065458,0.003148,NaN,0
35622,243064k,Heimfeld,28,18,2016-12-15 07:36:26,2016-12-15 07:37:03,HFS,S3,2016-12-15,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",27,0.028197,0.018888,NaN,0
35623,243064k,Harburg-Rathaus,70,60,2016-12-15 07:38:42,2016-12-15 07:39:30,HRF,S3,2016-12-15,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",24,0.070493,0.062959,NaN,0
35624,243064k,Harburg,131,20,2016-12-15 07:40:31,2016-12-15 07:41:25,HRS,S3,2016-12-15,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",23,0.131923,0.020986,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610665,243064k,Diebsteich,2,8,2017-03-31 08:13:03,2017-03-31 08:13:41,DT,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",15,0.002014,0.008395,-6.0,0
610666,243064k,Langenfelde,0,0,2017-03-31 08:14:53,2017-03-31 08:15:28,LST,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",40,0.000000,0.000000,0.0,0
610667,243064k,Stellingen,0,2,2017-03-31 08:16:49,2017-03-31 08:17:28,STS,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",58,0.000000,0.002099,-2.0,0
610668,243064k,Eidelstedt,0,0,2017-03-31 08:19:00,2017-03-31 08:19:41,EST,S3,2017-03-31,"Neugraben,Neuwiedenthal,Heimfeld,Harburg-Ratha...",17,0.000000,0.000000,0.0,0


In [1]:
df.loc[(df['TrainID'] == '243064k') & (df['ArrivalDate'] ==  datetime.date(2017,3,31))]

NameError: name 'df' is not defined

In [29]:
emission_probabilities = {}

#for trainid in s1['TrainID'].unique():
for station in s1['Station'].unique():
        # For boarding
    station_boardings = s1[s1['Station'] == station]['Boardings']
    total_boardings = station_boardings.sum()
    boarding_probabilities = station_boardings.value_counts() / total_boardings
        
        # For alighting
    station_alightings = s1[s1['Station'] == station]['Alightings']
    total_alightings = station_alightings.sum()
    alighting_probabilities = station_alightings.value_counts() / total_alightings
        
        # Combine boarding and alighting into emission probabilities for the station
    emission_probabilities[station] = {
        'boarding': boarding_probabilities.to_dict(),
        'alighting': alighting_probabilities.to_dict()
        }

    # Display the emission probabilities
    # 
print(emission_probabilities)

{'Wedel': {'boarding': {0: 0.03170370895304237, 9: 0.000859251122784328, 11: 0.0008326763457909982, 12: 0.0008149598277954451, 16: 0.0008061015687976685, 8: 0.0008061015687976685, 10: 0.0007706685328065622, 7: 0.0007529520148110091, 13: 0.0007263772378176793, 15: 0.0007086607198221261, 14: 0.0006820859428287964, 6: 0.0006643694248332433, 30: 0.0006466529068376902, 21: 0.0006377946478399136, 18: 0.000628936388842137, 31: 0.0005580703168599244, 19: 0.0005492120578621478, 5: 0.0005492120578621478, 17: 0.0005403537988643712, 29: 0.0005403537988643712, 32: 0.0005226372808688181, 23: 0.0005226372808688181, 20: 0.0005049207628732648, 26: 0.0005049207628732648, 22: 0.0004960625038754883, 25: 0.00047834598587993517, 35: 0.00046948772688215856, 24: 0.00046948772688215856, 4: 0.0004340546908910523, 34: 0.00042519643189327567, 33: 0.00040747991389772256, 37: 0.00039862165489994595, 3: 0.0003897633959021694, 28: 0.00038090513690439284, 2: 0.00037204687790661623, 39: 0.00036318861890883967, 40: 0.00

In [30]:
# Convert the emission probabilities to a matrix format expected by hmmlearn
emission_matrix = np.array([np.hstack((emission_probabilities[station]['boarding'], 
                                       emission_probabilities[station]['alighting']))
                            for station in s1['Station'].unique()])

In [46]:
s1["TrainID"].unique()[:3]

array(['247283', '247306', '237363'], dtype=object)

In [50]:
s1.loc[(s1['TrainID'] == i)]['ArrivalDate'].unique()

array([datetime.date(2016, 12, 10), datetime.date(2016, 12, 11),
       datetime.date(2016, 12, 18), datetime.date(2017, 1, 3),
       datetime.date(2016, 12, 30), datetime.date(2016, 12, 28),
       datetime.date(2016, 12, 26), datetime.date(2017, 1, 11),
       datetime.date(2017, 1, 13), datetime.date(2017, 1, 9),
       datetime.date(2017, 1, 25), datetime.date(2017, 1, 26),
       datetime.date(2017, 2, 2), datetime.date(2017, 2, 4),
       datetime.date(2017, 2, 5), datetime.date(2017, 1, 27),
       datetime.date(2017, 2, 11), datetime.date(2017, 2, 19),
       datetime.date(2017, 2, 17), datetime.date(2017, 2, 20),
       datetime.date(2017, 2, 24), datetime.date(2017, 2, 28),
       datetime.date(2017, 3, 4), datetime.date(2017, 3, 22),
       datetime.date(2017, 3, 28), datetime.date(2017, 3, 31)],
      dtype=object)

In [54]:
s1['on_train'] = 0
s1['going_next_station'] = 0

In [66]:
for i in s1["TrainID"].unique():
    for date in s1.loc[(s1['TrainID'] == i)]['ArrivalDate'].unique():
        s1.loc[(s1['TrainID'] == i) & (s1['ArrivalDate'] == date), 'on_train'] = s1.loc[(s1['TrainID'] == i) & (s1['ArrivalDate'] == date)]['Boardings'] - s1.loc[(s1['TrainID'] == i) & (s1['ArrivalDate'] == date)]['Alightings']
        s1.loc[(s1['TrainID'] == i) & (s1['ArrivalDate'] == date), 'going_next_station'] = s1.loc[(s1['TrainID'] == i)  & (s1['ArrivalDate'] == date)]['on_train'].cumsum()


In [83]:
grouped = s1.groupby(['TrainID', 'ArrivalDate'])
s1['on_train'] = grouped['Boardings'].transform(lambda x: x) - grouped['Alightings'].transform(lambda x: x)
s1['going_next_station'] = grouped['on_train'].cumsum()

C:\Users\kfilonch.SVA\AppData\Local\Temp\ipykernel_6688\1714968654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1['on_train'] = grouped['Boardings'].transform(lambda x: x) - grouped['Alightings'].transform(lambda x: x)
C:\Users\kfilonch.SVA\AppData\Local\Temp\ipykernel_6688\1714968654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1['going_next_station'] = grouped['on_train'].cumsum()


In [84]:
s1

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,on_train,going_next_station
611,247283,Wedel,89,0,2016-12-10 11:39:06,2016-12-10 11:43:29,WL,S1,2016-12-10,89,89
612,247283,Rissen,17,2,2016-12-10 11:47:17,2016-12-10 11:47:50,RI,S1,2016-12-10,15,104
613,247283,Sülldorf,14,8,2016-12-10 11:50:32,2016-12-10 11:52:07,SDF,S1,2016-12-10,6,110
614,247283,Iserbrook,6,0,2016-12-10 11:53:21,2016-12-10 11:53:57,IS,S1,2016-12-10,6,116
615,247283,Blankenese,12,7,2016-12-10 11:55:54,2016-12-10 11:57:31,B,S1,2016-12-10,5,121
...,...,...,...,...,...,...,...,...,...,...,...
610620,247587k,Friedrichsberg,8,12,2017-03-30 22:45:01,2017-03-30 22:45:33,FBG,S1,2017-03-30,-4,65
610621,247587k,Barmbek,13,26,2017-03-30 22:47:20,2017-03-30 22:48:00,BAP,S1,2017-03-30,-13,52
610622,247587k,Alte Wöhr,0,23,2017-03-30 22:49:32,2017-03-30 22:50:08,AW,S1,2017-03-30,-23,29
610623,247587k,Rübenkamp,0,7,2017-03-30 22:51:08,2017-03-30 22:51:49,RP,S1,2017-03-30,-7,22


In [51]:
s1.loc[(s1['TrainID'] == '247283') & (s1['ArrivalDate'] == datetime.date(2016, 12, 10))]['Boardings'] - s1.loc[(s1['TrainID'] == '247283') & (s1['ArrivalDate'] == datetime.date(2016, 12, 10))]['Alightings']

611     89
612     15
613      6
614      6
615      5
616      2
617      3
618     39
619      4
620     83
621      1
622      8
623     -7
624     -5
625   -119
626    -35
627      7
628     -2
629      3
630     -4
631     -5
632      5
633     -3
634     -9
635    -23
636    -82
dtype: int32

In [65]:
s1.loc[s1['TrainID'] == '247283']

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,on_train,going_next_station
611,247283,Wedel,89,0,2016-12-10 11:39:06,2016-12-10 11:43:29,WL,S1,2016-12-10,89.0,89.0
612,247283,Rissen,17,2,2016-12-10 11:47:17,2016-12-10 11:47:50,RI,S1,2016-12-10,15.0,104.0
613,247283,Sülldorf,14,8,2016-12-10 11:50:32,2016-12-10 11:52:07,SDF,S1,2016-12-10,6.0,110.0
614,247283,Iserbrook,6,0,2016-12-10 11:53:21,2016-12-10 11:53:57,IS,S1,2016-12-10,6.0,116.0
615,247283,Blankenese,12,7,2016-12-10 11:55:54,2016-12-10 11:57:31,B,S1,2016-12-10,5.0,121.0
...,...,...,...,...,...,...,...,...,...,...,...
608612,247283,Barmbek,44,40,2017-03-31 12:36:50,2017-03-31 12:37:27,BAP,S1,2017-03-31,4.0,193.0
608613,247283,Alte Wöhr,3,5,2017-03-31 12:39:00,2017-03-31 12:39:25,AW,S1,2017-03-31,-2.0,191.0
608614,247283,Rübenkamp,12,22,2017-03-31 12:40:30,2017-03-31 12:41:12,RP,S1,2017-03-31,-10.0,181.0
608615,247283,Ohlsdorf,38,63,2017-03-31 12:42:57,2017-03-31 12:45:05,OPS,S1,2017-03-31,-25.0,156.0


In [34]:
aggregated_data = s1.groupby('Station').sum(['Boardings', 'Aightings']).reset_index()

# Calculate the continuing passengers and transition probabilities
aggregated_data['Continuing'] = aggregated_data['Boardings'] - aggregated_data['Alightings'].shift(-1).fillna(0)
aggregated_data['Transition_Prob'] = aggregated_data['Continuing'] / aggregated_data['Boardings']

# Create the transition matrix for the train line
n_stations = len(aggregated_data)
transition_matrix = np.zeros((n_stations, n_stations))

for i in range(n_stations - 1):
    transition_matrix[i, i + 1] = aggregated_data.iloc[i]['Transition_Prob']

# The last station has no further stations, so we set its probability to stay there as 1
transition_matrix[-1, -1] = 1

print(transition_matrix)

[[ 0.00000000e+00  7.61132504e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -6.77317319e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.0

In [35]:
aggregated_data

,Station,Boardings,Alightings,on_train,going_next_station,Continuing,Transition_Prob
0,Airport,217359,209702,0.0,0.0,165439.0,0.761133
1,Alte Wöhr,53618,51920,-1.0,115.0,-363164.0,-6.773173
2,Altona,448703,416782,-2.0,64.0,299044.0,0.666463
3,Bahrenfeld,142372,149659,-13.0,51.0,-104165.0,-0.731640
4,Barmbek,229827,246537,-8.0,107.0,30281.0,0.131756
5,Berliner Tor 1,179986,199546,-30.0,120.0,933.0,0.005184
6,Blankenese,174414,179053,4.0,26.0,88195.0,0.505665
7,Friedrichsberg,89215,86219,8.0,115.0,-39472.0,-0.442437
8,Hasselbrook,133206,128687,29.0,151.0,-434957.0,-3.265296
9,Hauptbahnhof,549896,568163,-40.0,80.0,514115.0,0.934931


In [71]:
s1

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,on_train,going_next_station
611,247283,Wedel,89,0,2016-12-10 11:39:06,2016-12-10 11:43:29,WL,S1,2016-12-10,89.0,89.0
612,247283,Rissen,17,2,2016-12-10 11:47:17,2016-12-10 11:47:50,RI,S1,2016-12-10,15.0,104.0
613,247283,Sülldorf,14,8,2016-12-10 11:50:32,2016-12-10 11:52:07,SDF,S1,2016-12-10,6.0,110.0
614,247283,Iserbrook,6,0,2016-12-10 11:53:21,2016-12-10 11:53:57,IS,S1,2016-12-10,6.0,116.0
615,247283,Blankenese,12,7,2016-12-10 11:55:54,2016-12-10 11:57:31,B,S1,2016-12-10,5.0,121.0
...,...,...,...,...,...,...,...,...,...,...,...
610620,247587k,Friedrichsberg,8,12,2017-03-30 22:45:01,2017-03-30 22:45:33,FBG,S1,2017-03-30,-4.0,65.0
610621,247587k,Barmbek,13,26,2017-03-30 22:47:20,2017-03-30 22:48:00,BAP,S1,2017-03-30,-13.0,52.0
610622,247587k,Alte Wöhr,0,23,2017-03-30 22:49:32,2017-03-30 22:50:08,AW,S1,2017-03-30,-23.0,29.0
610623,247587k,Rübenkamp,0,7,2017-03-30 22:51:08,2017-03-30 22:51:49,RP,S1,2017-03-30,-7.0,22.0


In [70]:
passengers_to_next = s1.loc[s1[''], 'going_next_station']

KeyError: 1

In [77]:
grouped = s1.groupby(['TrainID', 'ArrivalDate'])
s1['on_train'] = grouped['Boardings'].transform(lambda x: x) - grouped['Alightings'].transform(lambda x: x)
s1['going_next_station'] = grouped['on_train'].cumsum()

KeyError: 'ArrivalDate'

In [115]:
grouped = s1.groupby(['TrainID', 'ArrivalDate'])

# Function to calculate transition probabilities within each group
def calculate_transition_probabilities(group):


    # Ensure there are no negative values
    group['going_next_station'] = group['going_next_station'].clip(lower=0)

    # Calculate the transition probabilities
    group['Transition_Prob'] = group['going_next_station']  / group['Alightings'].shift(-1).fillna(0) 
    group['Transition_Prob'] = group['Transition_Prob'].fillna(0).clip(upper=1)

    # Handle the last station (set transition probability to 1)
    group.iloc[-1, group.columns.get_loc('Transition_Prob')] = 1

    return group

# Apply the function to each group
transition_prob = grouped.apply(calculate_transition_probabilities)

In [88]:
transition_prob

TrainID         Station  Boardings  Alightings  \
TrainID ArrivalDate                                                          
237033n 2017-01-14  203730  237033n      Blankenese          3           0   
                    203731  237033n        Hochkamp          0           4   
                    203732  237033n  Klein Flottbek          2           1   
                    203733  237033n     Othmarschen          1           0   
                    203734  237033n      Bahrenfeld          0           0   
...                             ...             ...        ...         ...   
247733n 2017-03-26  568503  247733n        Ohlsdorf          2           8   
                    568504  247733n         Kornweg          0           0   
                    568505  247733n     Hoheneichen          0           0   
                    568506  247733n  Wellingsbüttel          2           3   
                    568507  247733n    Poppenbüttel          0          18   

                                       Arrival           Departure  \
TrainID ArrivalDate                                                  
237033n 2017-01-14  203730 2017-01-14 03:27:03 2017-01-14 03:37:10   
                    203731 2017-01-14 03:39:21 2017-01-14 03:40:07   
                    203732 2017-01-14 03:41:49 2017-01-14 03:42:08   
                    203733 2017-01-14 03:43:53 2017-01-14 03:44:13   
                    203734 2017-01-14 03:46:12 2017-01-14 03:47:13   
...                                        ...                 ...   
247733n 2017-03-26  568503 2017-03-26 03:43:47 2017-03-26 03:46:36   
                    568504 2017-03-26 03:48:47 2017-03-26 03:49:20   
                    568505 2017-03-26 03:50:20 2017-03-26 03:51:04   
                    568506 2017-03-26 03:52:35 2017-03-26 03:53:18   
                    568507 2017-03-26 03:55:10 2017-03-26 04:09:37   

                           StationNameShort sBahnID ArrivalDate  on_train  \
TrainID ArrivalDate                                                         
237033n 2017-01-14  203730                B      S1  2017-01-14         3   
                    203731              HPS      S1  2017-01-14        -4   
                    203732               FB      S1  2017-01-14         1   
                    203733               OH      S1  2017-01-14         1   
                    203734              BAF      S1  2017-01-14         0   
...                                     ...     ...         ...       ...   
247733n 2017-03-26  568503              OPS      S1  2017-03-26        -6   
                    568504              KWS      S1  2017-03-26         0   
                    568505              HCH      S1  2017-03-26         0   
                    568506              WBS      S1  2017-03-26        -1   
                    568507               PB      S1  2017-03-26       -18   

                            going_next_station  Transition_Prob  
TrainID ArrivalDate                                              
237033n 2017-01-14  203730                   3         1.000000  
                    203731                   0         1.000000  
                    203732                   0         0.000000  
                    203733                   1         0.000000  
                    203734                   1         1.000000  
...                                        ...              ...  
247733n 2017-03-26  568503                   9         0.000000  
                    568504                   9         0.000000  
                    568505                   9         0.333333  
                    568506                   8         1.000000  
                    568507                   0         1.000000  

[272260 rows x 12 columns]

In [121]:
aggregated_data = s1.groupby('Station').sum(['Alightings', 'going_next_station']).reset_index()

# Calculate the continuing passengers and transition probabilities
#aggregated_data['Continuing'] = aggregated_data['Boardings'] - aggregated_data['Alightings'].shift(-1).fillna(0)
aggregated_data['Transition_Prob'] =  aggregated_data['Alightings'] / aggregated_data['going_next_station'] 

# Create the transition matrix for the train line
n_stations = len(aggregated_data)
transition_matrix = np.zeros((n_stations, n_stations))

for i in range(n_stations - 1):
    transition_matrix[i, i + 1] = aggregated_data.iloc[i]['Transition_Prob']

# The last station has no further stations, so we set its probability to stay there as 1
transition_matrix[-1, -1] = 1

print(transition_matrix)

[[0.         1.02325604 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.04954232 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.38164089 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.14810992 0.
  0.         

In [116]:
t  = transition_prob.reset_index(drop=True)

In [117]:
t = t.groupby(['TrainID', 'ArrivalDate'])

In [118]:
# Get a list of all unique stations
stations = s1['Station'].unique()
n_stations = len(stations)
station_indices = {station: idx for idx, station in enumerate(stations)}

# Initialize an array to store all transition matrices
all_transition_matrices = []

# Function to create a transition matrix for each group
def create_transition_matrix(group):
    # Initialize transition matrix for this group
    transition_matrix = np.zeros((n_stations, n_stations))

    # Iterate over the rows of the group
    prev_station = None
    for _, row in group.iterrows():
        if prev_station is not None:
            # Calculate indices for the previous and current station
            i, j = station_indices[prev_station], station_indices[row['Station']]
            transition_matrix[i, j] = row['Transition_Prob']
        prev_station = row['Station']

    return transition_matrix

# Apply the function to each group and accumulate the matrices
for _, group in t:
    matrix = create_transition_matrix(group)
    all_transition_matrices.append(matrix)

# Calculate the average transition matrix
average_transition_matrix = np.mean(all_transition_matrices, axis=0)
#sum_transition_matrix = np.sum(all_transition_matrices, axis=0)
# Normalize the transition matrix so each row sums to 1
#äscaler=MinMaxScaler()
#normalized_transition_matrix =scaler.fit(average_transition_matrix)

print("Normalized Average Transition Matrix:")
print(average_transition_matrix)


Normalized Average Transition Matrix:
[[4.73888731e-04 3.24271656e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 8.51884683e-04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.32688845e-03]
 [3.29352668e-01 0.00000000e+00 3.26053092e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.83937927e

In [114]:
print(scaler.transform(normalized_transition_matrix))

TypeError: float() argument must be a string or a real number, not 'MinMaxScaler'

In [73]:
# Initialize the transition matrix with zeros
transition_matrix = np.zeros((n_stations, n_stations))

# Calculate the transition probabilities
for i in s1['Station'].unique():
    # The number of passengers from station i to i+1
    passengers_to_next = s1.loc[s1['Station'] == i, 'going_next_station']
    print(passengers_to_next)
    
    # The total number of passengers leaving station i
    total_passengers_leaving = passengers_to_next  # For a train route this is typically the same
    
    # For a train, we assume passengers only go to the next station in the sequence
    if total_passengers_leaving > 0:
        transition_matrix[i, i+1] = passengers_to_next / total_passengers_leaving
    else:
        transition_matrix[i, i+1] = 0

# The last station does not transition anywhere, passengers alight
transition_matrix[-1, -1] = 1

print(transition_matrix)

611       89.0
662      -38.0
663       69.0
814      -11.0
815       12.0
          ... 
608862   -90.0
610497   -19.0
610498    12.0
610555   -17.0
610556    12.0
Name: going_next_station, Length: 7044, dtype: float64


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
# Initialize the transition matrix
stations = s1['Station'].unique()
transition_matrix = pd.DataFrame(0, index=stations, columns=stations)

# Calculate transition probabilities
for i in range(len(aggregated_data) - 1):
    current_station = aggregated_data.iloc[i]['Station']
    next_station = aggregated_data.iloc[i + 1]['Station']
    passengers_to_next = df.iloc[i]['Continuing']
    if i > 0:
        passengers_from_prev = df.iloc[i - 1]['Continuing']
    else:
        passengers_from_prev = passengers_to_next  # Handle the first row
    
    transition_prob = passengers_to_next / passengers_from_prev if passengers_from_prev > 0 else 0
    transition_matrix.loc[current_station, next_station] = transition_prob

# Normalize the transition matrix so that each row sums to 1
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)

KeyError: 'Pass going to next'